In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Homework1

Mounted at /gdrive
/gdrive/My Drive/Homework1


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import matplotlib.pyplot as plt
from random import randint

In [3]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [4]:
seed = 34
import os
import random

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [5]:
def load_data(folder="public_data.npz", resolution=96, head_only=False):
    images = []

    loaded = np.load(folder, allow_pickle=True)

    # Iterate through files in the specified folder
    for i, img in enumerate(loaded['data']):
        # Normalize image pixel values to a float range [0, 1]
        #img = (img / 255).astype(np.float32)

        # Convert image from BGR to RGB
        #img = img[...,::-1]

        # Make the image dataset squared
        dim = min(img.shape[:-1])
        img = img[(img.shape[0]-dim)//2:(img.shape[0]+dim)//2, (img.shape[1]-dim)//2:(img.shape[1]+dim)//2, :]

        # Resize the image to 224x224 pixels
        #img = tfkl.Resizing(224, 224)(img)
        img = tfkl.Resizing(resolution, resolution)(img)

        if img is not None:
            images.append(img)

        if (head_only and i == 9):
           break

    labels = loaded['labels']
    loaded.close()

    if (head_only):
       labels = labels[:10]

    y = LabelEncoder().fit_transform(labels)
    y = tfk.utils.to_categorical(y, 2)

    return np.array(images), y




def display_random_images(X, y, num_img=10):
  # Create subplots for displaying items
  fig, axes = plt.subplots(2, num_img//2, figsize=(20, 9))
  for i in range(num_img):
      image = randint(0, X.shape[0] - 1)

      ax = axes[i%2, i%num_img//2]
      ax.imshow(np.clip(X[image], 0, 255))  # Display clipped item images
      ax.text(0.5, -0.1, str(image) + ' ' + str(y[image]), size=12, ha="center", transform=ax.transAxes)
      ax.axis('off')
  plt.tight_layout()
  plt.show()




def delete_outliers(X, y):
  shrek = 137
  trololo = 5143

  new_X = []
  new_y = []

  num_outliers = 0

  for i, sample in enumerate(X):
    if (not (np.array_equal(sample, X[shrek]) or np.array_equal(sample, X[trololo]))):
      new_X.append(sample)
      new_y.append(y[i])
    else:
      num_outliers += 1

  return np.array(new_X), np.array(new_y), num_outliers

In [6]:
X, y = load_data('public_data.npz')
X, y, num_outliers = delete_outliers(X, y)

In [7]:
# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=600, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train_0, y_val = train_test_split(X_train_val, y_train_val, test_size=600, stratify=np.argmax(y_train_val,axis=1))

print(X_train.shape, y_train_0.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(3804, 96, 96, 3) (3804, 2)
(600, 96, 96, 3) (600, 2)
(600, 96, 96, 3) (600, 2)


In [8]:
augment1 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomTranslation(height_factor = (-0.2,0.2), width_factor = (-0.2,0.2), fill_mode = 'reflect'),
    tfkl.RandomZoom(0.3, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.1, value_range=(0,255)),
])

augment2 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomZoom(0.4, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.1, value_range=(0,255)),
    tfkl.RandomRotation((-1,1), fill_mode = 'reflect'),
])

new_X_train_1 = augment2(X_train[np.where((y_train_0[:, 0] == 0) & (y_train_0[:, 1] == 1))])
augmented_X_train_2 = augment1(X_train)
augmented_X_train_1 = augment2(augmented_X_train_2)

X_train = np.append(X_train ,augmented_X_train_2, axis = 0)
X_train = np.append(X_train ,augmented_X_train_1, axis = 0)
X_train = np.append(X_train ,new_X_train_1, axis = 0)

y_train = np.append(y_train_0, y_train_0, axis = 0)
y_train = np.append(y_train, y_train_0, axis = 0)
for k in range(new_X_train_1.shape[0]):
    y_train = np.append(y_train, [[0,1]], axis = 0)

In [9]:
# Define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
batch_size = 16                # Batch size for training, always a power of 2!!
epochs = 400

In [10]:
mobile = tfk.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
mobile.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [11]:
inputs = tfk.Input(shape=input_shape)
# Connect MobileNetV2 to the input
x = mobile(inputs)
x = tfkl.Dropout(rate = 1/5)(x)
x = tfkl.Dense(64)(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
x = tfkl.Dense(16)(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
model_1 = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
model_1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=1e-3, weight_decay=3e-5), metrics=['accuracy'])

# Display model summary
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 mobilenetv2_1.00_96 (Funct  (None, 1280)              2257984   
 ional)                                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 64)                81984     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 64)                0     

In [12]:
history = model_1.fit(
    x = preprocess_input(X_train), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (preprocess_input(X_val), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
804/804 [==============================] - 28s 18ms/step - loss: 0.5851 - accuracy: 0.6954 - val_loss: 0.4241 - val_accuracy: 0.7983 - lr: 0.0010
Epoch 2/400
804/804 [==============================] - 11s 14ms/step - loss: 0.5096 - accuracy: 0.7422 - val_loss: 0.3861 - val_accuracy: 0.8167 - lr: 0.0010
Epoch 3/400
804/804 [==============================] - 10s 12ms/step - loss: 0.4869 - accuracy: 0.7610 - val_loss: 0.3885 - val_accuracy: 0.8217 - lr: 0.0010
Epoch 4/400
804/804 [==============================] - 12s 15ms/step - loss: 0.4748 - accuracy: 0.7688 - val_loss: 0.3785 - val_accuracy: 0.8300 - lr: 0.0010
Epoch 5/400
804/804 [==============================] - 13s 16ms/step - loss: 0.4620 - accuracy: 0.7781 - val_loss: 0.3719 - val_accuracy: 0.8317 - lr: 0.0010
Epoch 6/400
804/804 [==============================] - 10s 13ms/step - loss: 0.4535 - accuracy: 0.7854 - val_loss: 0.3828 - val_accuracy: 0.8217 - lr: 0.0010
Epoch 7/400
804/804 [==============================]

In [13]:
# Evaluate the model on the test set
test_accuracy = model_1.evaluate(preprocess_input(X_test),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

Test set accuracy 0.8567


In [14]:
model_1.save('mobilenetv2_and_augmentation_NO_INVERSION')

In [15]:
del model_1

In [10]:
ft_model = tfk.models.load_model('mobilenetv2_and_augmentation_NO_INVERSION')

In [11]:
ft_model.get_layer('mobilenetv2_1.00_96').trainable = True

In [12]:
print(len(ft_model.get_layer('mobilenetv2_1.00_96').layers))

155


In [13]:
ft_model.get_layer('mobilenetv2_1.00_96').summary()

Model: "mobilenetv2_1.00_96"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 48, 48, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 48, 48, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 48, 48, 32)           0         ['bn_Conv1[0

In [14]:
# Freeze first N layers
N = 12    # last block
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_96').layers[:-N]):
  layer.trainable=False

In [15]:
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_96').layers):
  print(layer.name, layer.trainable)

input_1 False
Conv1 False
bn_Conv1 False
Conv1_relu False
expanded_conv_depthwise False
expanded_conv_depthwise_BN False
expanded_conv_depthwise_relu False
expanded_conv_project False
expanded_conv_project_BN False
block_1_expand False
block_1_expand_BN False
block_1_expand_relu False
block_1_pad False
block_1_depthwise False
block_1_depthwise_BN False
block_1_depthwise_relu False
block_1_project False
block_1_project_BN False
block_2_expand False
block_2_expand_BN False
block_2_expand_relu False
block_2_depthwise False
block_2_depthwise_BN False
block_2_depthwise_relu False
block_2_project False
block_2_project_BN False
block_2_add False
block_3_expand False
block_3_expand_BN False
block_3_expand_relu False
block_3_pad False
block_3_depthwise False
block_3_depthwise_BN False
block_3_depthwise_relu False
block_3_project False
block_3_project_BN False
block_4_expand False
block_4_expand_BN False
block_4_expand_relu False
block_4_depthwise False
block_4_depthwise_BN False
block_4_depthwi

In [16]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(), metrics='accuracy')

In [17]:
history = ft_model.fit(
    x = preprocess_input(X_train), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (preprocess_input(X_val), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
804/804 [==============================] - 34s 19ms/step - loss: 0.5536 - accuracy: 0.7442 - val_loss: 0.5673 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 2/400
804/804 [==============================] - 13s 17ms/step - loss: 0.4371 - accuracy: 0.7976 - val_loss: 0.4089 - val_accuracy: 0.8483 - lr: 0.0010
Epoch 3/400
804/804 [==============================] - 13s 17ms/step - loss: 0.4014 - accuracy: 0.8179 - val_loss: 0.4607 - val_accuracy: 0.8267 - lr: 0.0010
Epoch 4/400
804/804 [==============================] - 13s 16ms/step - loss: 0.3769 - accuracy: 0.8315 - val_loss: 0.4318 - val_accuracy: 0.8517 - lr: 0.0010
Epoch 5/400
804/804 [==============================] - 12s 15ms/step - loss: 0.3381 - accuracy: 0.8532 - val_loss: 0.4166 - val_accuracy: 0.8400 - lr: 0.0010
Epoch 6/400
804/804 [==============================] - 13s 17ms/step - loss: 0.3137 - accuracy: 0.8667 - val_loss: 0.4119 - val_accuracy: 0.8267 - lr: 0.0010
Epoch 7/400
804/804 [==============================]

In [20]:
ft_model.evaluate(preprocess_input(XXX),y_test)

19/19 [==============================] - 3s 80ms/step - loss: 0.7239 - accuracy: 0.8217


[0.7238536477088928, 0.8216666579246521]

In [19]:
XXX = tf.identity(X_test)

In [18]:
X_test

array([[[[ 74., 124.,  39.],
         [ 54., 103.,  21.],
         [ 59., 105.,  30.],
         ...,
         [128., 148.,  87.],
         [126., 146.,  85.],
         [141., 161., 100.]],

        [[ 79., 129.,  44.],
         [ 61., 110.,  28.],
         [ 60., 104.,  29.],
         ...,
         [136., 156.,  97.],
         [145., 165., 106.],
         [162., 182., 121.]],

        [[ 73., 118.,  37.],
         [ 68., 112.,  33.],
         [ 62., 106.,  31.],
         ...,
         [135., 156., 100.],
         [112., 133.,  77.],
         [157., 178., 121.]],

        ...,

        [[151., 184., 115.],
         [151., 184., 115.],
         [151., 184., 113.],
         ...,
         [153., 184., 125.],
         [145., 176., 119.],
         [150., 181., 124.]],

        [[148., 181., 112.],
         [148., 181., 112.],
         [145., 178., 107.],
         ...,
         [139., 170., 110.],
         [141., 172., 115.],
         [132., 163., 106.]],

        [[142., 175., 106.],
       

In [21]:
ft_model.save('mobilenetv2_finetuned_NO_INVERSION')